<a id=index></a>
# Index
- <a href=#s3>S3</a>
- <a href=#cloudfront>CloudFront</a>
- <a href=#ec2>EC2</a>

<a id=s3></a>
## S3
<a href=#index>Go to Index</a>

### Generic info
- Object based storage (not block storage)
- ACL are used to control access to specific resources. Bucket policy applies to the whole bucket.
- When a file is uploaded to S3, the browser gets a 200 response for a successful upload. 
- All objects are private by default
- Min sized object on S3 is 0 bytes (eg: Delete marker)
- 3 types of storage - Standard, IA (Infrequently accessed), Reduced Redundancy Storage
- Can use client side or server side encryption. Server side encryption consists of 3 options -
	- S3 (SSE-S3)
	- KMS (SSE-KMS)
	- Customer provided keys (SSE-C)
- S3 Standard: 11 9s of durability and 4 9s of availability and can stand the loss of two facilities concurrently
- S3 IA: Data that is infrequently retrieved but needs to be accessible quickly when needed. Cheaper than Standard but charged per retrieval
- 'S3 One Zone - Infrequently Accessed' is replacing Reduced Redundancy Storage (RRS)
    - For data that need not be stored in multiple AZ and is infrequently accessed
    - Availability is 99.5%
- Glacier: Is for data archival. 3 types of retrieval (decreasing order of cost, increasing order of time)
    - Expedited
    - Standard
    - Bulk
- S3 buckets can be configured to create access logs (can dump all logs to another bucket)
- S3 uploads can get much faster if done as multi-part upload
- Can have upto 100 buckets per account by default
- Single operation upload to S3 works for files <5GB. Multi-part works for files from 5MB to 5TB.

### Consistency
- Read after Write consistency for new object PUTS (instantaneous)
- Eventual consistency for overwrite PUTS and DELETE actions (can take a little time to propagate)

### Versioning
- For a public file with versioning enabled, it will be made private in the new version. 
- Versioning cannot be removed (can be suspended)
- Actually when you delete a file, it gets another version called 'delete marker'. If you delete the delete marker, the file gets restored.

- The files only get removed when you got to versioning and remove the versions themselves
- There is also an option to enable MFA for deleting files with versioning enabled.

### Cross Region Replication (CRR)
- When CRR is newly introduced on a bucket, the existing objects are not automatically copied. New and updated objects are copied to the replication bucket. AWS CLI (s3 cp --recursive) can be used to copy all files from the primary to the replication bucket
- Versioning must be enabled (source and target) for CRR
- Src and target should be in different regions (Cross **Region** replication)
- At this time, you cannot do replication to multiple buckets (daisy chaining is not supported at present)
- Deleting files (delete marker) or deletion of individual versions are not replicated to the secondary buckets (this is an AWS security feature

### Lifecycle rule
- Main idea is to reduce cost
- S3 can automatically move contents of a bucket (or folder or things with a spefied tags ) to S3-IA and later to S3 Glacier based on what we set in the Management>Lifecycle tab added as Lifecycle rule. - Can also be configured to permanently delete stuff at some point
- To move to S3-IA requires a minimum of 30 days after creation and moving to Glacier requires 30 days after residing in IA or 60 days from creation
- Can be done along with versioning.

### S3 Security & Encryption
- By default all S3 objects are private
- Use SSL/TLS (https) for encryption during transit
- At rest use one of three server side encryption options 
    - S3 Managed Keys (SSE-S3)
    - S3 KMS (charges you, tells you who is decrypting what when)
    - Customer provided key (SSE-C). You handle the keys
- Can also choose to do client side encryption

### Storage Gateway
Is a service that connects local on-prem IT network with a cloud storage for scalable and cost-effective storage. It can connect to the on-prem storage and automatically replicate things on to S3 or S3 Glacier.

There are 4 types of storage gateways -
1. File Gateway (NFS) - for media files, flat files, etc.. stored directly on S3
2. Volume Gateways (iSCSI):  for block based storage. Think of these as a connection to harddisks(disk **volumes**). Data written to these disk volumes can be backed up using EBS. These can be used to save system snapshots. These snapshots are incremental (only changes get captured). Snapshot storage is also compressed to minimize storage costs.
    - Stored Volumes: Entire dataset is stored on site and is asynchronously backed up to S3
    - Cached Volumes: Entire data is stored on S3 and most frequently accessed data is cached on site
3. Virtual Tape Library Gateway (VTL): Used for backup. Uses popular applications like NetBackup, Backup exec, Veenam etc.
    

### Snowball

Legacy option was Import/Export Disk (portable disks) that Amazon allowed for large scale data transfer. 

Snowball can import to and export from **S3**.
- Snowball: 256 AES encryption, tamper proof, multiple layer security, trusted platform module. comes with a kindle to track the device. Software erase after each transfer (so that no one can retrieve data from it)
- Snowball Edge: like Snowball, but also has compute capacity. Eg: Airline manufacturer can place this device in a test flight plane for data capture. Like an mobile AWS datacenter in a box.
- Snowmobile: Petabytes or exabytes of data. Truck trailer. Even a complete datacenter migration.

### S3 Transfer Acceleration
Instead of uploading drectly to an S3 bucket in some region directly, upload to an edge location that is close to you.

Within S3 Properties>Transfer acceleration. Provides a new endpoint.

### S3 Static Website Hosting
- The public endpoing follows a specific format "<bucket-name>-s3-website-<region>-amazonaws.com"
- Use bucket policy to make entire S3 bucket public
- S3 scales automatically to meet demand for the website


<a id=cloudfront></a>
## CloudFront
<a href=#index>Go to Index</a>

### Generic
- CDN - Content Delivery Network. Something that enables faster content delivery of content if the geographical location of the requestor and requested content might otherwise cause delay in service.
- Edge location (EL) is different from AWS region or AZ. Currently there are over 50 ELs. 
- Edge location takes content from origin. At this time origin can be 
	- S3 bucket
	- EC2 instance
	- Elastic Load Balancer
	- Route53
	- Can also be an external source of the files
- Distribution is the name given to the CDN which consists of a collection of Edge locations
- When content is requested first time, it is fetched from origin and then cached in the EL. This is cached for a specified TTL (Time To Live)
- There are two types of distribution
	- Web Distribution - Typically used for websites
	- RTMP - used for media streaming
- Edge locations are **not read-only**. They can also be used for put operations and that object will then be written back to the origin.
- You can clear the cached objects but you will be charged for it. The normal behavior is that the cached object will expire on its own after the specified TTL.
- Specifics of setting up the CloudFront Distribution
    - CloudFront uses an identity to access the S3 bucket (Origin Access Identity parameter in the CloudFront dist setup)
    - In the setup, remember to 'Grant Read permissions on bucket' to 'yes'
    - Allow HTTP and HTTPS(recommended for regular pages)
    - TTL is an important design consideration (set based on how frequently the objects change)
    - Pick the default SSL certificate so as to support HTTPS unless you can publish a certificate
    - There is a feature called Restrictions - allows Geo-Restrictions using a Whitelist and a Blacklist. Blacklist allows global access and restricts to few entities. Whitelist only serves to the entities in the whitelist.
    - "Invalidation" is the process of removing an object that is cached on CloudFront. Remember that this is to delete objects before TTL. *This costs money.*

<a id="ec2">
# EC2: Elastic Cloud Compute
<a href="#index">Back to Index</a>

Provides resizeable compute capacity in the cloud: 
    - can scale up and down (increase/decrese capacity by starting new instances in minutes)
    - can scale out (Elastic load balancer)
    
EC2 can be provisioned -
- On Demand: 
    - Pay fixed rate by the hour - Windows (or by the second - Linux)
    - Good for temporary needs (Dev)
- Reserved instances: 
    - Contracts with Amazon 1 or 3 year contracts. Significant dicount (75% off of on-demand prices)
    - Great for predictible usage
        - Standard RIs (upto 70% off on-demand)
        - Convertible RIs (upto 54% off on-demand) allow you to change the attributes of the RIs as long as it creates instances of equal or higher value
        - Scheduled RIs: good for a known pattern in demand/requests.
- Spot Instances: 
    - By at a price that you are comfortable with (like a market)
    - Flexible start and end price
    - Applications that are only feasible at low compute prices
    - If Spot instance is terminated by Amazon EC2, you will not be charged for a partial hour of usage. However, if you terminate the instance yourself, you will be charged for the complete hour in which the instance ran.
- Dedicated Hosts: 
    - A physical machine for you. Eg: If your license is attached to a mac id, you can use it on cloud by hosting it in a Dedicated host. 
    - It could be because of regulatory requirements too
    - Can be purchased on-demand and can also be reserved (upto 70% off on-demand price)

Pneumonic to remember all instance types: FIGHT DR.Mc PX. There are instances that star with all these letters. Think Edward Norton fights (Fight) Dr. McDonald (Dr.Mc) who is from Scotland and hands out pictures (PX) of Scotlant

### EBS
Elastic Block Storage. If EC2 are servers online, EBS are disks online. EBS is placed in a specific AZ and replicated within that AZ.
Comes in different kinds - 
    - General purpose SSD (GP2): 
        - balances price and performance
        - speed of 3 to 10k IOPS per GB. And the ability to burst upto 3000 IOPS for extended periods of time for volumes 3334 GB and above
   - Provisioned SSD IOPS:
       - For I/O intensive applications like large relational or NoSQL databases
       - if you need more than 10,000 IOPS
       - Can provision up to 20,000 IOPS
   - Throughput Optimized HDD (ST1) (magnetic disk):
       - Cannot be boot volume. Can only be additional volumes (eg: cannot be C drive in Windows)
       - Good for big data, data warehousing, log processing etc
   - Cold HDD (SC1) (magnetic disk)
       - Lowest cost for infrequently accessed workloads
       - File server
       - Cannot be boot volume
   - Magnetic (Standard) (legacy):
       - Lowest cost per GB of all EBS bootable vol types
       - Ideal for worlkoads where data is accessed infrequently
       - For applications where the lowest storage cost is important


### EC2 Hands-on session
- Tenancy is the property that enables you to use a dedicated host or a shared host
- Remember that one Subnet (within a VPC) falls in one AZ
- Monitoring property during EC2 allows for detailed monitoring (RAM, CPU utilzation etc. every 1 min). Default is to track these every 5 mins
- Delete on Termination is the default behavior for EBS volumens attached to EC2 instances
- There are two checks that are listed on EC2 console
    - System status check: Ensures network connectivity to the instance is working fine. This is equivalent to checking that the Hypervisor level things are working alright
    - Instance status check: Ensures the OS is able to receive packages 
    
- By default, the root volume cannot be encrypted. It can be encrypted by creating a copy and encrypting it and being used as root volume.
- Termination protection is turned off by default
- On EBS backed instance, the default action is for the root EBS volume to be deleted when instance is terminated
- EBS root volumes of your default AMIs cannot be encrypted. Can use a third party tool such as bit locker to encrypt root volumes. 
- Additional EBS volumes can be encrypted
- Terminated instances will be visible in the list of instances for 1 hour after termination
    

### Security Groups
- Any change that is made to an SG takes effect immediately
- SGs are STATEFUL (in contrast with a Network Access Control list). Meaning if you allow inbound rules, then those will also receive outbound traffic even if there are no rules mentioned in the outbound section of the SG.
- You cannot deny access of specific kind or for a specific IP address using SGs. You can only 'allow' using SGs. There are no inbound rules by default
- All outbound traffic is allowed by default
- Can attach multiple SGs to an instance
- Can have any number of EC2 instances within an SG

### EBS volumes
- EBS volumes have to be in the same AZ as the EC2 instance
- To make a copy of EBS in a different AZ, make a snapshot and create volume in required AZ using snapshot
- Use the Actions>Copy option to create the instance in another Region
- Except 'Standard' (primitive magnetic), all other volumes type and sizes can be edited after creation. No associated downtime
- EBS Snapshots are created for backups and EBS images are used to clone EC2 instances
- Root volume is deleted when EC2 instance is terminated. Other EBS volumes are not terminated
- Can take snapshots when instance is running except if you're taking snapshots of EBS volumes that serve as root volumes
- Snapshot of encrypted volumes are encrypted. Volumes restored from encrypted snapshots are encrypted. 
- Only snapshots of unencrypted snapshots can be shared publicly

Volumes exist on EBS. Snapshots exist on S3. Snapshots are incremental (only changes from last snapshot are captured)

### AMI 
AMI is selected based on 
- Region
- OS
- Architecture
- Launch Permissions
- Storage for the Root Device
- Storage for Root Device - Instance Store (Ephemeral Storage) or EBS backed volumes
    - Remember that instance store cannot have 'start'/'stop' but EBS backed has these. When they are started after being stopped, it starts in a different hypervisor
    - Instance store backed instances will lose data if the underlying host fails
    - EBS volumes can be detached from one instance and attached to another. This cannot be done for Instance based volumes
    - Both types can be rebooted without loss of data
    
### Elastic Load Balancer
Come in 3 types - 
- Application load balancer: Very advanced, can send specific request to specific web servers, work at layer 7 of TCP model
- Network load balancer: For extreme performance, works at connection layer (layer 4). Can handle millions of requests per sec with ultra low latency
- Classic load balancer: These are legacy load balancers (what are meant when Elastic Load Balancer is mentioned). Can balance HTTP/HTTPS requests and also use layer-7 specific features (X-forwarded-For and sticky sessions). Can also use strict layer 4 LB for applications that rely strictly on TCP protocol. 

X-Forwarded-For header can be used to pass the public ip (IPv4) of the requester to the EC2 instance from the ELB

If the application stops responding ELB responds with Error code 504 (Gateway Timed Out). Could be because the Web Server or the Database Server failed

ELBs do healthchecks for EC2 instances and keeps track of "InService" or "OutOfService"

### Cloudwatch
Can be used to monitor components and also set alarms to notify about events of interest
- Standard monitoring: 5 mins
- Detailed monitoring: 1 mins

**Remember that Cloudwatch is for monitoring and CloudTrail is for audit. CloudTrail captures everything that happens within the AWS account**